# CV part one

В этой тетрадке мы рассмотрим задачу распознавания лиц на примере датасета [CelebA](http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html)

**Предполагаем, что ноутбук запущен внутри Yandex DataSphere**

In [5]:
#!M
from pathlib import Path

import numpy as np
import pandas as pd
from tqdm import tqdm
import cv2

import torch
import torch.nn as nn
# import torch.nn.functional as F
# from torch import optim
from torch.utils.data import Dataset, DataLoader, Sampler
from torchvision.models import resnet34
# from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics.pairwise import euclidean_distances
import torch.nn.functional as F
from collections import Counter
from torch.utils.data import TensorDataset

## Data

Качаем архив с данными с Yandex Object Storage и распаковываем в текущую папку.

Структура архива:
- /celeba_data/
    - train.csv
    - val.csv
    - images/{image}.jpg

CSV файлы содержат название файла (`image`) и его лейбл (`label`).

In [2]:
#!M
from cloud_ml.storage.api import Storage

s3 = Storage.s3(access_key="Le9tg70HQEJsoGqjqXH8", secret_key="NV75mCPkC0PEd35ImyDI5vI7p40YGFOYZgkH7moa")
# downloading contents of the remote file into the local one
s3.get('dl-hse-2021/celeba_data.zip', './celeba_data.zip')

In [3]:
#!:bash
unzip -q ./celeba_data.zip -d ./ && rm celeba_data.zip

replace ./celeba_data/images/052628.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: error:  invalid response [#]
replace ./celeba_data/images/052628.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL
(EOF or read error, treating as "[N]one" ...)


## Задание 1
**(0.2 балла)** Напишите класс датасет, который будет возвращать картинку и ее лейбл.

In [6]:
#!M
class CelebADataset(Dataset):
    def __init__(self, images_dir_path: str,
                 description_csv_path: str):
        super().__init__()
        
        self.images_dir_path = images_dir_path
        self.description_df = pd.read_csv(description_csv_path,
                                           dtype={'image_name': str, 'label': int})
        
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, item):
        img, label = self.description_df.iloc[item, :]
        
        img_path = Path(self.images_dir_path, f'{img}')
        img = cv2.imread(str(img_path.resolve()))
        
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
       
        img = img.astype(np.float32) / 255.0 # img \in [0, 1]
        mean = np.array([0.485, 0.456, 0.406]).reshape(1, 1, 3)
        std =  np.array([0.229, 0.224, 0.225]).reshape(1, 1, 3)
        img = (img - mean) / std
        img = img.astype(np.float32)
        img = np.transpose(img, [2, 0, 1])[None, ...]
        img = torch.tensor(img)
                
        return dict(
            sample=img,
            label=label,
        )

In [7]:
#!M
train = CelebADataset(images_dir_path='celeba_data/images/',
                           description_csv_path='celeba_data/train.csv')
val = CelebADataset(images_dir_path='celeba_data/images/',
                         description_csv_path='celeba_data/val.csv')

## Задание 2
**(0.2 балла)** Напишите функцию, которая будет считать метрику top-n accuracy.

$$TopN \ Accuracy = \frac{Number \ of \ objects \ with \ correct \ answer \ among \ topN \ predictions}{Total \ number \ of \ objects}$$

*Example:*

![image](https://www.baeldung.com/wp-content/ql-cache/quicklatex.com-ae746981c7a437b7e1fc2831e5d76d57_l3.svg)  
$Top3 \ Accuracy = \frac{4}{5} = 0.8$

*Hint:* Для каждого объекта выбираем `n` наиболее уверенных предсказаний. Если среди них есть правильный ответ, то увеличиваем числитель и знаменатель на единицу, иначе увеличиваем только знаменатель.

In [8]:
#!M
def top_n_accuracy(preds: np.ndarray,
                   targets: np.ndarray,
                   n_size: int) -> float:
    preds = preds.T[0:n_size].T
    num = 0
    
    for i in range(len(targets)):
        if targets[i] in preds[i]:
            num += 1
            
    return num/len(targets)

## Задание 3
**(0.2 балла)** Решите задачу без дообучения.

*Step-by-step:*
1. Инициализируйте предобученную сетку (`backbone`).
1. Прогоните через нее все картинки из валидационного датасета и сложите полученные эмбеддинги в массив.
1. Для каждого вектора найдите ближайшие к нему векторы и отсортируйте их по расстоянию (cosine, euclidian, ...). Лейблы соседних векторов будут предсказаниями для текущего вектора.
1. Оставьте топ-5 уникальных предсказаний.
1. Посчитайте и выведите метрики:
    1. top-1 accuracy
    1. top-5 accuracy

*Вопросы:*
1. Зачем мы заменяем последний линейный слой на `Identity` ?
1. Зачем используем на сетке метод `eval` ?

*Hints:*
1. Для расчета попарных расстояний лучше не использовать циклы, а считать все в матрицах. Описание подхода к расчету L2 расстояний: [link](https://math.stackexchange.com/questions/3147549/compute-the-pairwise-euclidean-distance-matrix)
1. Так можно использовать sklearn реализации: [link](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics.pairwise)
1. Для получения top-k предсказаний не обязательно сортировать весь массив.

Ответы на вопросы:
1. Потому что иначе мы получим вероятности принадлежности к классу. А нас интересуют не вероятности, а возможность узнать лейблы N ближайших соседей, чтобы определить есть ли истинный лейбл среди них.
2. Чтобы прогнать тензоры черезь сеть без ее тренировки, нужен метод eval. Если dropout и batch normalization слои не в режиме eval, это приводит к искажению результатов.  

In [9]:
#!M
backbone = resnet34(pretrained=True)
backbone.fc = nn.Identity()
backbone = backbone.eval()

val_loader = DataLoader(val, shuffle=False, batch_size=1)

vectors = []
labels = []
with torch.no_grad():
    for data in val_loader.dataset:
        x = data['label']
        labels.append(x)
        y =  backbone(data['sample'])
        y = np.array(y).flatten()
        vectors.append(y)
        
distances = euclidean_distances(vectors)
np.fill_diagonal(distances, 10**10)

preds = []
for j in range(distances.shape[0]):
    distance_label = dict(zip(distances[j],labels))
    list_keys = list(distance_label.keys())
    list_keys.sort()
    k = 0
    for i in list_keys:
        k+=1
        preds.append(distance_label[i])
        if k > 30:
            break
            
n = np.array(labels).shape[0]
preds = np.array(preds).reshape((n, k))

top5 = []
for i in range(n):
    top5_1 = []
    for j in preds[i]:
        
        if j not in top5_1:
            top5_1.append(j)
        if len(top5_1) == 5:
            break
    top5.append(top5_1)

print('top5', top_n_accuracy(np.array(top5), np.array(labels), 5))
print('top1', top_n_accuracy(np.array(top5), np.array(labels), 1))

top5 0.28952534353450443
top1 0.18125534806462978


/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'str'>
Walking trough too many objects
Use %enable_full_walk to serialize all variables correctly
  {name: self._state[name] for name in self._state.varnames() if not self._skip_variable(name)}
/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'dict'> with size: 19784
Use %enable_full_walk to serialize all variables correctly
  {name: self._state[name] for name in self._state.varnames() if not self._skip_variable(name)}
/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'list'> with size: 19784
Use %enable_full_walk to serialize all variables correctly
  {name: self._state[name] for name in self._state.varnames() if not self._skip_variable(name)}
/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'list'> with size: 19867
Use %enable_full_

## Задание 4
**(0.4 балла)** Решите задачу с дообучением на эмбеддингах.

*Step-by-step:*
1. Напишите небольшую сетку произвольной архитектуры, которая будет использовать эмбеды, выдаваемые `backbone` сетью.
1. Напишите класс Dataset, который будет возвращать эмбединг и лейбл.
1. Напишите класс Sampler [PyTroch docs](https://pytorch.org/docs/stable/data.html#data-loading-order-and-sampler), который будет отвечать за правильность сбора тренировочных батчей: якорный пример, позитивный, негативный.
1. Обучите ее на тренировочном датасете:
    1. Лосс -- [triplet loss](https://pytorch.org/docs/stable/generated/torch.nn.TripletMarginLoss.html).
    1. Метрика -- top-5 accuracy.
1. Посчитайте top-1 и top-5 accuracy на валидации. Насколько сильно они отличаются от того, что получилось в предыдущем задании?


*Hints:*
1. Убедитесь, что у каждого лейбла есть как минимум 2 примера, иначе не получится достать позитивный пример.
1. Лучше предварительно прогнать все картинки из трейна и сохранить полученные эмбеддинги, чтобы при обучении сети грузить только эмбеды (векторы).

In [12]:
#!M
backbone = resnet34(pretrained=True)
backbone.fc = nn.Identity()
backbone = backbone.eval()

train_loader = DataLoader(train, shuffle=False, batch_size=1)

train_vectors = []
train_labels = []
with torch.no_grad():
    for data in train_loader.dataset:
        x = data['label']
        train_labels.append(x)
        y =  backbone(data['sample'])
        y = np.array(y).flatten()
        train_vectors.append(y)

/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'list'> with size: 162770
Use %enable_full_walk to serialize all variables correctly
  {name: self._state[name] for name in self._state.varnames() if not self._skip_variable(name)}
/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'list'> with size: 162770
Use %enable_full_walk to serialize all variables correctly
  {name: self._state[name] for name in self._state.varnames() if not self._skip_variable(name)}
/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'list'> with size: 19867
Use %enable_full_walk to serialize all variables correctly
  {name: self._state[name] for name in self._state.varnames() if not self._skip_variable(name)}
/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'str'>
Walking trough too many objects
Use %enable_ful

In [60]:
#!XL
# создание датасетов 
train_dataset = TensorDataset(torch.tensor(train_vectors), torch.tensor(train_labels))
val_dataset = TensorDataset(torch.tensor(vectors), torch.tensor(labels))       
    
#Вместо самплера
#------------------------------------
def get_anchor_positive_mask(labels):

    # Check that i and j are distinct
    indices_equal = torch.eye(labels.size(0)).bool()
    indices_not_equal = ~indices_equal

    # Check if labels[i] == labels[j]
    # Uses broadcasting where the 1st argument has shape (1, batch_size) and the 2nd (batch_size, 1)
    labels_equal = labels.unsqueeze(0) == labels.unsqueeze(1)

    return labels_equal & indices_not_equal


def get_anchor_negative_mask(labels):

    # Check if labels[i] != labels[k]
    # Uses broadcasting where the 1st argument has shape (1, batch_size) and the 2nd (batch_size, 1)

    return ~(labels.unsqueeze(0) == labels.unsqueeze(1))

def get_triplet_mask(labels):

    mask_anchor_positive = get_anchor_positive_mask(labels)
    mask_anchor_negative = get_anchor_negative_mask(labels)

    return mask_anchor_positive.unsqueeze(2) & mask_anchor_negative.unsqueeze(1)

def batch_triplet_loss(embeddings, labels):
    
    pairwise_dist = torch.cdist(embeddings, embeddings, p = 2)
    
    mask_anchor_positive = get_anchor_positive_mask(labels).float()
    anchor_positive_dist = pairwise_dist * mask_anchor_positive
    hardest_positive_dist, _ = anchor_positive_dist.max(1, keepdim=True)

    
    mask_anchor_negative = get_anchor_negative_mask(labels).float()
    anchor_negative_dist = pairwise_dist + 999. * (1.0 - mask_anchor_negative)
    hardest_negative_dist, _ = anchor_negative_dist.min(1, keepdim=True)

    triplet_loss = hardest_positive_dist - hardest_negative_dist + params["margin"]
    
    triplet_loss[triplet_loss < 0] = 0
    return torch.mean(triplet_loss)
#------------------------------------    

class convEmbedding(nn.Module):
    def __init__(self, c_dim = 1):
        super().__init__()

        self.embedding = nn.Sequential(nn.Flatten(), nn.Linear(512, 2048), nn.ReLU(), nn.Linear(2048, 512))


    def forward(self, input_tensor):
        x = self.embedding(input_tensor)
        return F.normalize(x, p =2, dim = 1)
    
model = convEmbedding()

def collate(batch):
    data = [item[0].view(1, 16,32) for item in batch]
    data = torch.stack(data).float()

    target = [item[1] for item in batch]
    target = torch.LongTensor(target)
    return data, target

def train_one_batch(model, x, y):
    x = x

    batch_embeddings = model(x)
    loss = batch_triplet_loss(batch_embeddings, y)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss.item()


params = {"learning_rate": 0.01, "margin": 0.5, "batch_size":  128, "epochs": 50} 

trainloader = DataLoader(train_dataset, batch_size = params["batch_size"], collate_fn = collate, drop_last = True)
steps_per_epoch  = train_dataset.__len__()// params["batch_size"]

model = convEmbedding()
optimizer = torch.optim.SGD(model.parameters(), params['learning_rate'])

  
train_losses = []
model.train()
for epoch in range(1, params["epochs"]+1):
    print('-' * 10)
    print('Epoch {}/{}\t{} batches'.format(epoch, params["epochs"], steps_per_epoch))
    
    curr_loss = []
    for step, (x, y) in enumerate(trainloader):
        loss = train_one_batch(model, x, y)
        curr_loss.append(loss)
        print('\rprogress {:6.1f} %\tloss {:8.8f}'.format(100*(step+1)/steps_per_epoch, np.mean(curr_loss)), end = "")
        
    train_losses.append(np.mean(curr_loss))
    print('\rprogress {:6.1f} %\tloss {:8.8f}'.format(100*(step+1)/steps_per_epoch, np.mean(curr_loss)))
   

model.eval()

val_loader = DataLoader(val_dataset, batch_size = 128, collate_fn = collate)

#!M
X = []; val_labels = []
for x, label in val_loader:
    batch_embedings = model(x)
    X.append(batch_embedings.cpu().detach().numpy())
    val_labels.append(label.numpy())
    

X = np.concatenate(X)
val_labels = np.concatenate(val_labels)
distances = euclidean_distances(X)
np.fill_diagonal(distances, 10**10)

preds = []
for j in range(distances.shape[0]):
    distance_label = dict(zip(distances[j],val_labels))
    list_keys = list(distance_label.keys())
    list_keys.sort()
    k = 0
    for i in list_keys:
        k+=1
        preds.append(distance_label[i])
        if k > 30:
            break
            
n = np.array(val_labels).shape[0]
preds = np.array(preds).reshape((n, k))

top5 = []
for i in range(n):
    top5_1 = []
    for j in preds[i]:
        
        if j not in top5_1:
            top5_1.append(j)
        if len(top5_1) == 5:
            break
    top5.append(top5_1)
print('----------')
print('top5', top_n_accuracy(np.array(top5), np.array(val_labels), 5))
print('top1', top_n_accuracy(np.array(top5), np.array(val_labels), 1))

----------
Epoch 1/50	1271 batches


progress  100.0 %	loss 0.01196944
----------
Epoch 2/50	1271 batches


progress  100.0 %	loss 0.01140376
----------
Epoch 3/50	1271 batches


progress  100.0 %	loss 0.01121165
----------
Epoch 4/50	1271 batches


progress  100.0 %	loss 0.01107452
----------
Epoch 5/50	1271 batches


...

progress  100.0 %	loss 0.00941961
----------
Epoch 50/50	1271 batches


progress  100.0 %	loss 0.00939906


----------


top5 0.3264710323652288
top1 0.20299994966527407


/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'list'> with size: 19867
Use %enable_full_walk to serialize all variables correctly
  {name: self._state[name] for name in self._state.varnames() if not self._skip_variable(name)}
/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'dict'> with size: 19767
Use %enable_full_walk to serialize all variables correctly
  {name: self._state[name] for name in self._state.varnames() if not self._skip_variable(name)}
/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'list'> with size: 19767
Use %enable_full_walk to serialize all variables correctly
  {name: self._state[name] for name in self._state.varnames() if not self._skip_variable(name)}
/kernel/lib/python3.7/site-packages/ipystate/state.py:135: UserWarning: Skipping walk through <class 'list'> with size: 162770
Use %enable_full_walk to seria

In [0]:
# your code must be before example

## Sampler (simple example)

В блоках ниже реализован пример датасета и сэмлера, который возвращает индексы для триплет лосса.

Датасет написан топорно, но основная логика следующая. Если ему на вход приходит `int`, то он возвращает название картинки (`img_name`) и ее лейбл (`img_label`). Если же приходит нечто длиной 3, то он возвращает 3 названия картинок, соответственно. В нашем случае это будет три картинки с двумя одинаковыми лейблами и одним другим: anchor, positive, negative.  
Сэмплер `SimpleTripletSampler`, в свою очередь, отвечает за формирование и поставку в датасет индексов триплетов.

Датасет и сэмлер объединяются внутри даталоадера.

*Hint:* Код написан только лишь для примера, поэтому логика возвращения триплетов может быть неверной.

In [0]:
class SimpleDataset(Dataset):
    def __init__(self, img_names: np.ndarray,
                 img_labels: np.ndarray):
        if len(img_names) != len(img_labels):
            raise ValueError('img_names and img_labels must have equal number of elements')

        self.img_names = img_names
        self.img_labels = img_labels

    def __len__(self):
        return len(self.img_names)
    
    def __getitem__(self, idx):
        if isinstance(idx, int):
            img_name = self.img_names[idx]
            img_label = self.img_labels[idx]
            
            return img_name, img_label
        else:
            assert len(idx) == 3
            
            anc_idx, pos_idx, neg_idx = idx
            anc_img_name = self.img_names[anc_idx]
            pos_img_name = self.img_names[pos_idx]
            neg_img_name = self.img_names[neg_idx]

            return anc_img_name, pos_img_name, neg_img_name


class SimpleTripletSampler(Sampler):
    def __init__(self, dataset: Dataset):
        super().__init__(dataset)

        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __iter__(self):
        for anchor_idx in range(len(self.dataset)):
            positive_idx = self._mine_positive(anchor_idx)
            negative_idx = self._mine_negative(anchor_idx)

            yield anchor_idx, positive_idx, negative_idx

    def _mine_positive(self, anchor_idx: int):
        labels: np.ndarray = self.dataset.img_labels

        anchor_label = labels[anchor_idx]
        pos_idxs = np.nonzero(labels == anchor_label)[0]
        pos_idx = np.random.choice(pos_idxs)

        return pos_idx

    def _mine_negative(self, anchor_idx: int):
        labels: np.ndarray = self.dataset.img_labels

        anchor_label = labels[anchor_idx]
        neg_idxs = np.nonzero(labels != anchor_label)[0]
        neg_idx = np.random.choice(neg_idxs)

        return neg_idx

In [0]:
ex_size = 100
np.random.seed(42)

# в нашем примере названием картинки будет выступать число от 0 до 99, а лейблом число от 0 до 4.
ex_dataset = SimpleDataset(img_names=np.arange(ex_size),
                           img_labels=np.random.randint(0, 5, size=ex_size))
ex_sampler = SimpleTripletSampler(dataset=ex_dataset)

ex_loader = DataLoader(dataset=ex_dataset, batch_size=10, sampler=ex_sampler)

In [0]:
# В этой ячейке мы дергаем первый батч с названиями картинок и достаем их лейблы, 
#  чтобы проверить действительно ли у них одинаковые или разные лейблы.
# Для тренировки сети с триплет лоссом сами лейблы нам не нужны будут.
#  Главное чтобы триплеты картинок формировались правильно: anchor, positive, negative

ex_batch = next(iter(ex_loader))

ex_batch_anc_labels = ex_dataset.img_labels[ex_batch[0]]
ex_batch_pos_labels = ex_dataset.img_labels[ex_batch[1]]
ex_batch_neg_labels = ex_dataset.img_labels[ex_batch[2]]

In [0]:
print('All anchor and positive labels are equal:', np.all(ex_batch_anc_labels == ex_batch_pos_labels))
print('Any of anchor and negative labels are equal:', np.any(ex_batch_anc_labels == ex_batch_neg_labels))

In [0]:
#!S
